In [27]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings

warnings.filterwarnings('ignore')
sns.set_style('white')

# Check GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using {device}')

# Prepare paths
results_path = Path('time_gan')
if not results_path.exists():
    results_path.mkdir()

experiment = 0
log_dir = results_path / f'experiment_{experiment:02}'
if not log_dir.exists():
    log_dir.mkdir(parents=True)

hdf_store = results_path / 'TimeSeriesGAN.h5'

# Parameters
seq_len = 24
n_seq = 2
batch_size = 128

# Data Normalization
df = pd.read_csv('output.csv')
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df).astype(np.float32)

# Create rolling window sequences
data = []
for i in range(len(df) - seq_len):
    data.append(scaled_data[i:i + seq_len])

n_windows = len(data)

# Create Dataset class
class TimeSeriesDataset(Dataset):
    def __init__(self, data):
        self.data = torch.tensor(data, dtype=torch.float32)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

real_series = DataLoader(TimeSeriesDataset(data), batch_size=batch_size, shuffle=True)

# Random data generator
def make_random_data():
    while True:
        yield np.random.uniform(low=0, high=1, size=(seq_len, n_seq))

random_series = DataLoader(TimeSeriesDataset(np.random.uniform(0, 1, (n_windows, seq_len, n_seq))), batch_size=batch_size, shuffle=True)

# RNN block generator
class RNNBlock(nn.Module):
    def __init__(self, n_layers, hidden_units, output_units):
        super(RNNBlock, self).__init__()
        self.gru = nn.GRU(input_size=n_seq, hidden_size=hidden_units, num_layers=n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_units, output_units)

    def forward(self, x):
        gru_out, _ = self.gru(x)
        out = torch.sigmoid(self.fc(gru_out))
        return out

# Model Components
hidden_dim = 2
num_layers = 3

embedder = RNNBlock(num_layers, hidden_dim, hidden_dim).to(device)
recovery = RNNBlock(num_layers, hidden_dim, n_seq).to(device)
generator = RNNBlock(num_layers, hidden_dim, hidden_dim).to(device)
supervisor = RNNBlock(2, hidden_dim, hidden_dim).to(device)
discriminator = RNNBlock(num_layers, hidden_dim, 1).to(device)

# Loss Functions
mse = nn.MSELoss()
bce = nn.BCELoss()

# Optimizers
autoencoder_optimizer = optim.Adam(list(embedder.parameters()) + list(recovery.parameters()))
supervisor_optimizer = optim.Adam(supervisor.parameters())
generator_optimizer = optim.Adam(list(generator.parameters()) + list(supervisor.parameters()))
discriminator_optimizer = optim.Adam(discriminator.parameters())
embedding_optimizer = optim.Adam(list(embedder.parameters()) + list(recovery.parameters()))

# Training Functions
def train_autoencoder(x):
    embedder.train()
    recovery.train()
    autoencoder_optimizer.zero_grad()
    h = embedder(x)
    x_tilde = recovery(h)
    loss_auto = mse(x, x_tilde)
    loss_auto.backward()
    autoencoder_optimizer.step()
    return loss_auto.item()

def train_supervisor(x):
    supervisor.train()
    supervisor_optimizer.zero_grad()
    h = embedder(x)
    h_hat = supervisor(h)
    loss_auto = mse(h[:, 1:], h_hat[:, :-1])
    loss_auto.backward()
    supervisor_optimizer.step()
    return loss_auto.item()

def get_discriminator_loss(real, fake):
    real_loss = bce(discriminator(real), torch.ones_like(discriminator(real)))
    fake_loss = bce(discriminator(fake), torch.zeros_like(discriminator(fake)))
    return real_loss + fake_loss

# def train_discriminator(real, fake):
#     discriminator.train()
#     discriminator_optimizer.zero_grad()
#     loss_d = get_discriminator_loss(real, fake)
#     loss_d.backward()
#     discriminator_optimizer.step()
#     return loss_d.item()

# def train_generator(real, fake):
#     generator.train()
#     supervisor.train()
#     generator_optimizer.zero_grad()
#     loss_g = bce(discriminator(supervisor(generator(fake))), torch.ones_like(discriminator(supervisor(generator(fake)))))
#     loss_g.backward()
#     generator_optimizer.step()
#     return loss_g.item()

# Training Loop
train_steps = 10000
for step in range(train_steps):
    X_ = next(iter(real_series))
    # print(X_.shape)
    X_ = X_.to(device)
    Z_ = next(iter(random_series))
    # print(Z_.shape)
    Z_ = Z_.to(device)

    ae_loss = train_autoencoder(X_)
    sup_loss = train_supervisor(X_)

    fake_data = supervisor(generator(Z_))
    # gen_loss = train_generator(X_, fake_data)
    generator.train()
    supervisor.train()
    generator_optimizer.zero_grad()
    loss_g = bce(discriminator(supervisor(generator(fake_data))), 
                 torch.ones_like(discriminator(supervisor(generator(fake_data)))))
    loss_g.backward()
    generator_optimizer.step()
    loss_g.item()
    # disc_loss = train_discriminator(X_, fake_data)
    discriminator.train()
    discriminator_optimizer.zero_grad()
    loss_d = get_discriminator_loss(X_, fake_data)
    loss_d.backward()
    discriminator_optimizer.step()
    loss_d.item()

    if step % 1000 == 0:
        print(f'Step {step}: AE Loss: {ae_loss}, Supervisor Loss: {sup_loss}, Gen Loss: {gen_loss}, Disc Loss: {disc_loss}')

# Generating synthetic data
generated_data = []
for _ in range(int(n_windows / batch_size)):
    Z_ = next(iter(random_series)).to(device)
    synthetic = recovery(supervisor(generator(Z_)))
    generated_data.append(synthetic.cpu().detach().numpy())

generated_data = np.array(np.vstack(generated_data))
generated_data = scaler.inverse_transform(generated_data.reshape(-1, n_seq)).reshape(-1, seq_len, n_seq)

# Save generated data
np.save(log_dir / 'generated_data.npy', generated_data)

# Plot sample series
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(14, 7))
axes = axes.flatten()

index = list(range(1, 25))
synthetic = generated_data[np.random.randint(n_windows)]
idx = np.random.randint(len(df) - seq_len)
real = df.iloc[idx: idx + seq_len]


sns.despine()
fig.tight_layout()
plt.show()

Using cuda


RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.